In [1]:

!pip install beautifulsoup4
!pip install xlrd
!pip install lxml
#file_location="C:\Users\tonya\OneDrive\Documents\CapstoneLatLong.xlsx"
import pandas as pd
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0--yes
import folium
%matplotlib inline
!pip install nbextensions



     |████████████████████████████████| 112kB 18.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 7.8MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 9.9MB/s eta 0:00:01     |█████████████████████▊          | 3.9MB 9.9MB/s eta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95

## Loading and Exploring Dataset

In [2]:
website_url=requests.get('https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population').text
from bs4 import BeautifulSoup
bsoup=BeautifulSoup(website_url,'lxml')
print(bsoup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Table of Atlanta neighborhoods by population - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XjuDpgpAAD8AAGQZnUAAAACB","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Table_of_Atlanta_neighborhoods_by_population","wgTitle":"Table of Atlanta neighborhoods by population","wgCurRevisionId":743264795,"wgRevisionId":743264795,"wgArticleId":34593265,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGro

In [3]:
My_table=bsoup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th><a class="mw-redirect" href="/wiki/Neighborhoods_of_Atlanta" title="Neighborhoods of Atlanta">Neighborhood</a></th>
<th>Population (2010)</th>
<th><a href="/wiki/Neighborhood_planning_unit" title="Neighborhood planning unit">NPU</a>
</th></tr>
<tr>
<td><a href="/wiki/Adair_Park" title="Adair Park">Adair Park</a></td>
<td>1,331</td>
<td>V
</td></tr>
<tr>
<td><a class="new" href="/w/index.php?title=Adams_Park_(Atlanta)&amp;action=edit&amp;redlink=1" title="Adams Park (Atlanta) (page does not exist)">Adams Park</a></td>
<td>1,763</td>
<td>R
</td></tr>
<tr>
<td><a class="mw-redirect" href="/wiki/Adamsville_(Atlanta)" title="Adamsville (Atlanta)">Adamsville</a></td>
<td>2,403</td>
<td>H
</td></tr>
<tr>
<td><a class="new" href="/w/index.php?title=Almond_Park&amp;action=edit&amp;redlink=1" title="Almond Park (page does not exist)">Almond Park</a></td>
<td>1,020</td>
<td>G
</td></tr>
<tr>
<td><a href="/wiki/Ansley_Park" title="Ansley Park">Ans

In [4]:
links=My_table.findAll('a')
links

[<a class="mw-redirect" href="/wiki/Neighborhoods_of_Atlanta" title="Neighborhoods of Atlanta">Neighborhood</a>,
 <a href="/wiki/Neighborhood_planning_unit" title="Neighborhood planning unit">NPU</a>,
 <a href="/wiki/Adair_Park" title="Adair Park">Adair Park</a>,
 <a class="new" href="/w/index.php?title=Adams_Park_(Atlanta)&amp;action=edit&amp;redlink=1" title="Adams Park (Atlanta) (page does not exist)">Adams Park</a>,
 <a class="mw-redirect" href="/wiki/Adamsville_(Atlanta)" title="Adamsville (Atlanta)">Adamsville</a>,
 <a class="new" href="/w/index.php?title=Almond_Park&amp;action=edit&amp;redlink=1" title="Almond Park (page does not exist)">Almond Park</a>,
 <a href="/wiki/Ansley_Park" title="Ansley Park">Ansley Park</a>,
 <a href="/wiki/Ardmore_(Atlanta)" title="Ardmore (Atlanta)">Ardmore</a>,
 <a class="mw-redirect" href="/wiki/Argonne_Forest_(Atlanta)" title="Argonne Forest (Atlanta)">Argonne Forest</a>,
 <a class="new" href="/w/index.php?title=Arlington_Estates&amp;action=edit&

In [5]:
Neighborhood=[]

In [6]:
for link in links:
    Neighborhood.append(link.get('title'))

In [7]:
print(Neighborhood)

['Neighborhoods of Atlanta', 'Neighborhood planning unit', 'Adair Park', 'Adams Park (Atlanta) (page does not exist)', 'Adamsville (Atlanta)', 'Almond Park (page does not exist)', 'Ansley Park', 'Ardmore (Atlanta)', 'Argonne Forest (Atlanta)', 'Arlington Estates (page does not exist)', 'Ashview Heights', 'Atlanta University Center', 'Atlantic Station', 'Audubon Forest (page does not exist)', 'Baker Hills (Atlanta) (page does not exist)', 'Bankhead (Atlanta)', 'Beecher Hills (page does not exist)', 'Ben Hill (Atlanta)', 'Ben Hill Terrace (page does not exist)', 'Benteen Park', 'Berkeley Park', 'Betmar LaVilla (page does not exist)', 'Bolton (Atlanta)', 'Boulevard Heights', 'Brandon (Atlanta) (page does not exist)', 'Brookhaven (Atlanta)', 'Brookwood (Atlanta)', 'Brookwood Hills', 'Browns Mill Park (page does not exist)', 'Buckhead Forest', 'Buckhead Village', 'Cabbagetown (Atlanta)', 'Campbellton Road (page does not exist)', 'Candler Park', 'Capitol Gateway (Atlanta) (page does not exis

In [8]:
import pandas as pd

# Data - Table of Atlanta Neighborhoods

In [9]:
df=pd.read_html('https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population')[0]

In [10]:
df

,Neighborhood,Population (2010),NPU
0,Adair Park,1331,V
1,Adams Park,1763,R
2,Adamsville,2403,H
3,Almond Park,1020,G
4,Ansley Park,2277,E
...,...,...,...
156,Westwood Terrace,733,I
157,Whittier Mill Village,617,D
158,Wildwood,1840,C
159,Wilson Mill Meadows,1096,H


#Use the csv file to create the geographical coordinates dataframe

In [11]:
import pandas as pd
df1=pd.read_csv("CapstoneLatLong_data.csv",encoding='latin1')
df1

,Neighborhood,Population,NPU,Latitude,Longitude
0,Adair Park,"1,331",V,33.731170,-84.410471
1,Adams Park,"1,763",R,33.714459,-84.461894
2,Adamsville,"2,403",H,33.760899,-84.503866
3,Almond Park,"1,020",G,33.781537,-84.467142
4,Ansley Park,"2,277",E,33.794558,-84.379037
...,...,...,...,...,...
154,West Paces Ferry/Northside,"1,218",A,33.735549,-84.450741
155,Westview,"3,020",T,33.808746,-84.484195
156,Westwood Terrace,733,I,33.814148,-84.418580
157,Whittier Mill Village,617,D,33.751908,-84.520908


In [12]:
address='Atlanta,Georgia'
geolocator=Nominatim(user_agent="atlanta_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Atlanta are {},{}.'.format(latitude,longitude))

The geographical coordinate of Atlanta are 33.7490987,-84.3901849.


# Map of Atlanta using latitude and longitude

In [13]:
map_atlanta=folium.Map(location=[33.7490987,-84.3901849],zoom_start=10)

In [14]:
for lat,lng,neighborhood in zip(df1['Latitude'],df1['Longitude'],
                                        df1['Neighborhood']):
    label='{}'.format(neighborhood)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_atlanta)
map_atlanta

In [15]:
Neighborhood_data=df1
Neighborhood_data

,Neighborhood,Population,NPU,Latitude,Longitude
0,Adair Park,"1,331",V,33.731170,-84.410471
1,Adams Park,"1,763",R,33.714459,-84.461894
2,Adamsville,"2,403",H,33.760899,-84.503866
3,Almond Park,"1,020",G,33.781537,-84.467142
4,Ansley Park,"2,277",E,33.794558,-84.379037
...,...,...,...,...,...
154,West Paces Ferry/Northside,"1,218",A,33.735549,-84.450741
155,Westview,"3,020",T,33.808746,-84.484195
156,Westwood Terrace,733,I,33.814148,-84.418580
157,Whittier Mill Village,617,D,33.751908,-84.520908


In [16]:
categorical_features=['Neighborhood','NPU']
continuous_features=['Population','Latitude','Longitude']


In [17]:
df1[continuous_features].describe()

,Latitude,Longitude
count,159.000000,159.000000
mean,33.762006,-84.415286
std,0.055608,0.057562
min,33.556652,-84.541877
25%,33.723828,-84.450271
50%,33.758025,-84.408500
75%,33.803277,-84.379541
max,33.873100,-84.081898


# Defining Foursquare Credentials

In [18]:

CLIENT_ID='CQBAZSAKMTHP315ZKWGJKX0JLHA1DTHNEU0MUM5M0AGJBDEA'
CLIENT_SECRET='QWII4IY420KMTUI0ION5GST3Z4IUSCSMPFK24GD0RMIQ0FGD'
VERSION='20200215'
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID:CQBAZSAKMTHP315ZKWGJKX0JLHA1DTHNEU0MUM5M0AGJBDEA
CLIENT_SECRET:QWII4IY420KMTUI0ION5GST3Z4IUSCSMPFK24GD0RMIQ0FGD


# Explore a neighborhood

In [19]:
Neighborhood_data.loc[0,'Neighborhood']
continuous_features=['Population','Latitude','Longitude']

# Get the neighborhoods latitude and longitude values

In [20]:
neighborhood_latitude=Neighborhood_data.loc[0,'Latitude']
neighborhood_longitude=Neighborhood_data.loc[0,'Longitude']
neighborhood_name=Neighborhood_data.loc[0,'Neighborhood']
print('Latitude and longitude values of {}, {},{}.'.format(neighborhood_name,neighborhood_latitude,neighborhood_longitude))

Latitude and longitude values of Adair Park, 33.73117,-84.410471.


# Get request Url

In [21]:
LIMIT=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                         CLIENT_SECRET,
                                                                                                                         VERSION,
                                                                                                                         neighborhood_latitude,
                                                                                                                         neighborhood_longitude,
                                                                                                                         radius,
                                                                                                                         LIMIT)

# Get Request and Check Results

In [22]:
results=requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5e3c294798205d001b604f00'},
 'response': {}}

# Extracts Category Venue

In [23]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

# Column of top venues

# Clean json into panda df

In [24]:
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

KeyError: 'groups'

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        #CREATE API REQUEST URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        #Get Request
        results=requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood',
                          'Neighborhood Latitude',
                          'Neighborhood Longitude',
                          'Venue',
                          'Venue Latitude',
                          'Venue Longitude',
                          'Venue Category']
    return(nearby_venues)  

# Run Function GetNearbyVenues

In [ ]:
neighborhood_venues=getNearbyVenues=getNearbyVenues(names=Neighborhood_data['Neighborhood'],
                                                                         latitudes=Neighborhood_data['Latitude'],
                                                                         longitudes=Neighborhood_data['Longitude'])

# Size of the DataFrame

In [ ]:
print(neighborhood_venues.shape)
neighborhood_venues.head()

In [ ]:
neighborhood_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

# Hot encoding

In [ ]:
Neighborhood_onehot=pd.get_dummies(neighborhood_venues[['Venue Category']],prefix="",prefix_sep="")
#add neighborhood column
Neighborhood_onehot['Neighborhood']=neighborhood_venues['Neighborhood']
#move neighborhood to first row
fixed_columns=[Neighborhood_onehot.columns[-1]] + list(Neighborhood_onehot.columns[:-1])
Neighborhood_onehot=Neighborhood_onehot[fixed_columns]

#Set number of clusters

In [ ]:
Neighborhood_onehot.head()

In [ ]:
Neighborhood_onehot.shape

# Group rows by neighborhood and mean of frequency

In [ ]:
Neighborhood_grouped=Neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
Neighborhood_grouped

In [ ]:
Neighborhood_grouped.shape

# 5 Most Common Venues

In [ ]:
num_top_venues=5
for hood in Neighborhood_grouped['Neighborhood']:
    print("****"+hood +"****")
    temp=Neighborhood_grouped[Neighborhood_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# 10 Venues

In [ ]:
num_top_venues=10
indicators=['st','nd','rd']
# Create Columns of top Venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
df3_venues_sorted=pd.DataFrame(columns=columns)
df3_venues_sorted['Neighborhood']=Neighborhood_grouped['Neighborhood']
for ind in np.arange(Neighborhood_grouped.shape[0]):
    df3_venues_sorted.iloc[ind,1:]=return_most_common_venues(Neighborhood_grouped.iloc[ind,:],num_top_venues)
df3_venues_sorted.head(40)

## Scatter Plot

In [ ]:

plt.scatter(df1['Latitude'],df1['Longitude'])

## Clusters

In [ ]:
km=KMeans(n_clusters=3)
km









































## Prediction Array

In [ ]:
y_predicted=km.fit_predict(df1[['Latitude','Longitude']])
y_predicted

## Use the Cluster 

In [ ]:
df1['cluster']=y_predicted
df1.head()

## Plot Clusters

In [ ]:
df2=df1[df1.cluster==0]
df3=df1[df1.cluster==1]
df4=df1[df1.cluster==2]

plt.scatter(df2.Latitude,df2['Longitude'],color='green')
plt.scatter(df3.Latitude,df3['Longitude'],color='red')
plt.scatter(df4.Latitude,df4['Longitude'],color='black')

plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.legend()

## Cluster Again

In [ ]:
km.cluster_centers_

In [ ]:
km=KMeans(n_clusters=3)
y_predicted=km.fit_predict(df1[['Latitude','Longitude']])
y_predicted

In [ ]:
df1['cluster']=y_predicted
df1

## Plot Cluster

In [ ]:
df2=df1[df1.cluster==0]
df3=df1[df1.cluster==1]
df4=df1[df1.cluster==2]

plt.scatter(df2.Latitude,df2['Longitude'],color='green')
plt.scatter(df3.Latitude,df3['Longitude'],color='red')
plt.scatter(df4.Latitude,df4['Longitude'],color='black')

plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.legend()

In [ ]:
k_rng=range(1,10)
sse=[]
for k in k_rng:
    km=KMeans(n_clusters=k)
    km.fit(df1[['Latitude','Longitude']])
    sse.append(km.inertia_)

In [ ]:
sse

## Plot Elbow Method

In [ ]:
plt.xlabel('K')
plt.ylabel('Sum of squared error')
plt.plot(k_rng,sse)

# Cluster Neighborhoods

# Set Number of Clusters

In [ ]:
kclusters=5
Neighborhood_grouped_clustering=Neighborhood_grouped.drop('Neighborhood',1)
#run K-means Clustering
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(Neighborhood_grouped_clustering)
#Check Cluster Labels
kmeans.labels_[0:10]


In [ ]:
#add clustering labels
df3_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)
Neighborhood_merged=Neighborhood_data
#merge data to longitude and latitude
Neighborhood_merged=Neighborhood_merged.join(df3_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Neighborhood_merged.head()


# Map

In [ ]:
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)
# set color
x=np.arange(kclusters)
ys=[i + x +(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
# add markers
markers_colors=[]
for lat,lon,poi,cluster in zip(Neighborhood_merged['Latitude'],Neighborhood_merged['Longitude'],Neighborhood_merged['Neighborhood'],Neighborhood_merged['Cluster Labels']):
    label=folium.Popup(str(poi) + 'Cluster ' + str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
       # color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

# Cluster 1

In [ ]:
Neighborhood_merged.loc[Neighborhood_merged['Cluster Labels']==0,Neighborhood_merged.columns[[1]+ list(range(5,Neighborhood_merged.shape[1]))]]

# Cluster 2

In [ ]:
Neighborhood_merged.loc[Neighborhood_merged['Cluster Labels']==1,Neighborhood_merged.columns[[1]+ list(range(5,Neighborhood_merged.shape[1]))]]

# Cluster 3

In [ ]:
Neighborhood_merged.loc[Neighborhood_merged['Cluster Labels']==2,Neighborhood_merged.columns[[1]+ list(range(5,Neighborhood_merged.shape[1]))]]

# Cluster 4

In [ ]:
Neighborhood_merged.loc[Neighborhood_merged['Cluster Labels']==3,Neighborhood_merged.columns[[1]+ list(range(5,Neighborhood_merged.shape[1]))]]

# Cluster 5

In [ ]:
Neighborhood_merged.loc[Neighborhood_merged['Cluster Labels']==4,Neighborhood_merged.columns[[1]+ list(range(5,Neighborhood_merged.shape[1]))]]

## End of Project